In [208]:
from math import factorial, floor, ceil

def print_table(arr, rows=[], columns=[]):
    if len(rows) > 0:
        for i in range(len(rows)):
            arr[i] = [str(rows[i])] + arr[i]

    if len(columns) > 0:
        arr = [[' '] + list(map(str, columns))] + arr

    row_lengths = [max(map(lambda x: len(str(x)), column)) for column in zip(*arr)]

    for row in arr:
        for i in range(len(row)):
            padding = row_lengths[i] - len(str(row[i])) + 2
            print(f'{" " * floor(padding / 2)}{row[i]}{" " * ceil(padding / 2)}', end='')
        print()

def calculate(lamb, mu, n, assistance=False):
    ro = lamb / mu
    p0 = 1 / sum((1 / factorial(k)) * ro ** k for k in range(n))

    p_cancelation = (1 / factorial(n) * (lamb / (n * mu))**n) * (p0 ** n if assistance else p0)
    # p_cancelation = 1 / (1 + sum([(ro ** k) / factorial(k) for k in range(n)]))
    Q = 1 - p_cancelation
    A = lamb * Q
    n_o = A / mu

    return [p_cancelation, Q, A, n_o]
    
n = 3
lamb = 4
mu = 2

no_assistance = calculate(lamb, mu, n)
unlimited_assistance = calculate(lamb, mu, n, assistance=True)
uniform_assistance = calculate(lamb, mu, n, assistance=True)

print_table([no_assistance, unlimited_assistance, uniform_assistance], 
            rows=['Без взаимопомощи', 'Неограниченная', 'Равномерная'], 
            columns=['Вероятность отказа', 'Относительная пропускная способность', 'Абсолютая пропускная способность', 'Среднее число занятых каналов'])

                    Вероятность отказа    Относительная пропускная способность  Абсолютая пропускная способность  Среднее число занятых каналов 
 Без взаимопомощи  0.009876543209876541            0.9901234567901235                  3.960493827160494                1.980246913580247       
  Неограниченная   0.0003950617283950617           0.9996049382716049                  3.9984197530864196              1.9992098765432098       
   Равномерная     0.0003950617283950617           0.9996049382716049                  3.9984197530864196              1.9992098765432098       


In [231]:
import simpy
import numpy as np

class QueueSystem:
    def __init__(self, env, channels, mutual_help=False, uniform_help=False):
        self.env = env
        self.channels = simpy.Resource(env, capacity=channels)
        self.mutual_help = mutual_help
        self.uniform_help = uniform_help

    def process_request(self, request):
        with self.channels.request() as req:
            yield req
            yield self.env.timeout(np.random.exponential(1 / request['service_rate']))

    def generate_requests(self, arrival_rate, service_rate):
        while True:
            yield self.env.timeout(np.random.exponential(1 / arrival_rate))
            request = {'service_rate': service_rate}
            self.env.process(self.process_request(request))

def simulate_no_help():
    env = simpy.Environment()
    system = QueueSystem(env, channels=3)

    env.process(system.generate_requests(arrival_rate=4, service_rate=0.5))

    env.run(until=1000)  # Set a simulation time limit

    total_requests = len(system.channels.users) + len(system.channels.queue)
    rejected_requests = total_requests - len(system.channels.queue)
    probability_of_rejection = rejected_requests / total_requests if total_requests > 0 else 0

    print(f"No help mode:")
    print(f"Total requests: {total_requests}")
    print(f"Rejected requests: {rejected_requests}")
    print(f"Probability of rejection: {probability_of_rejection:.5%}")

def simulate_unlimited_help():
    env = simpy.Environment()
    system = QueueSystem(env, channels=3, mutual_help=True)

    env.process(system.generate_requests(arrival_rate=4, service_rate=0.5))

    env.run(until=1000)

    total_requests = len(system.channels.users) + len(system.channels.queue)
    rejected_requests = total_requests - len(system.channels.queue)
    probability_of_rejection = rejected_requests / total_requests if total_requests > 0 else 0

    print(f"\nUnlimited help mode:")
    print(f"Total requests: {total_requests}")
    print(f"Rejected requests: {rejected_requests}")
    print(f"Probability of rejection: {probability_of_rejection:.5%}")

def simulate_uniform_help():
    env = simpy.Environment()
    system = QueueSystem(env, channels=3, mutual_help=True, uniform_help=True)

    env.process(system.generate_requests(arrival_rate=4, service_rate=0.5))

    env.run(until=1000)

    total_requests = len(system.channels.users) + len(system.channels.queue)
    rejected_requests = total_requests - len(system.channels.queue)
    probability_of_rejection = rejected_requests / total_requests if total_requests > 0 else 0

    print(f"\nUniform help mode:")
    print(f"Total requests: {total_requests}")
    print(f"Rejected requests: {rejected_requests}")
    print(f"Probability of rejection: {probability_of_rejection:.5%}")

# Uncomment the desired simulation to run
simulate_no_help()
simulate_unlimited_help()
simulate_uniform_help()


No help mode:
Total requests: 2568
Rejected requests: 3
Probability of rejection: 0.11682%

Unlimited help mode:
Total requests: 2515
Rejected requests: 3
Probability of rejection: 0.11928%

Uniform help mode:
Total requests: 2482
Rejected requests: 3
Probability of rejection: 0.12087%
